# A1111 Model Installer — SD 1.5 (UI)

This installs **SD 1.5** assets for Automatic1111 using a friendly UI:

- **Checkpoints** → `/workspace/a1111/models/Stable-diffusion/`
- **VAE (optional)** → `/workspace/a1111/models/VAE/`
- **ControlNet (SD15)** → `/workspace/a1111/models/ControlNet/`
- **Upscalers & Restorers** → `/workspace/a1111/models/(ESRGAN|GFPGAN|Codeformer)/`

Re-running is safe (existing files are skipped).  
If a model is gated on Hugging Face, set `HF_TOKEN` in your RunPod template.

In [ ]:
# First time? Uncomment, run once, then Kernel → Restart
# !pip install -q ipywidgets tqdm requests huggingface_hub
print('Downloader UI ready (install dependencies once if widgets are missing).')

In [ ]:
from pathlib import Path; import os
DATA_ROOT=Path(os.environ.get('WEBUI_ROOT','/workspace/a1111'))
CKPT_DIR=DATA_ROOT/'models/Stable-diffusion';VAE_DIR=DATA_ROOT/'models/VAE';CN_DIR=DATA_ROOT/'models/ControlNet'
ESRGAN_DIR=DATA_ROOT/'models/ESRGAN';GFPGAN_DIR=DATA_ROOT/'models/GFPGAN';CODEFORMER_DIR=DATA_ROOT/'models/Codeformer'
for d in (CKPT_DIR,VAE_DIR,CN_DIR,ESRGAN_DIR,GFPGAN_DIR,CODEFORMER_DIR): d.mkdir(parents=True,exist_ok=True)
print('Data root:',DATA_ROOT)

In [ ]:
import requests,shlex,subprocess,shutil,time;from tqdm import tqdm;from huggingface_hub import hf_hub_download
def human(n):
  for u in ['B','KB','MB','GB']:
    if n<1024:return f'{n:.1f} {u}';n/=1024
  return f'{n:.1f} TB'
def download_direct(url,dest):
  dest.parent.mkdir(parents=True,exist_ok=True)
  if dest.exists():print('✔ Exists:',dest);return dest
  r=requests.get(url,stream=True);r.raise_for_status();total=int(r.headers.get('Content-Length',0));tmp=dest.with_suffix(dest.suffix+'.part')
  with open(tmp,'wb') as f,tqdm(total=total if total>0 else None,unit='B',unit_scale=True,desc=dest.name,ncols=80) as bar:
    for ch in r.iter_content(1024*1024):
      if ch:f.write(ch);bar.update(len(ch) if total>0 else 0)
  tmp.replace(dest);print('✔ Saved:',dest,'('+human(dest.stat().st_size)+')');return dest
def download_hf(repo_id,filename,dest_dir,dest_name=None,token=None):
  dest=dest_dir/(dest_name or filename)
  if dest.exists():print('✔ Exists:',dest);return dest
  print('→ HF:',repo_id+'/'+filename)
  p=hf_hub_download(repo_id=repo_id,filename=filename,token=token or os.environ.get('HF_TOKEN'),local_dir=dest_dir,local_dir_use_symlinks=False,resume_download=True)
  if Path(p)!=dest:shutil.copy2(p,dest)
  print('✔ HF downloaded:',dest,'('+human(dest.stat().st_size)+')');return dest
print('Helpers loaded.')

## SD 1.5 models

In [ ]:
import ipywidgets as W;from IPython.display import display,Markdown
vae=W.Checkbox(description='Install SD1.5 MSE VAE',value=False)
cn=[('Canny','lllyasviel/control_v11p_sd15_canny'),('Depth','lllyasviel/control_v11p_sd15_depth'),('SoftEdge','lllyasviel/control_v11p_sd15_softedge'),('LineArt','lllyasviel/control_v11p_sd15_lineart'),('NormalBae','lllyasviel/control_v11p_sd15_normalbae'),('OpenPose','lllyasviel/control_v11p_sd15_openpose')]
pre={'Canny','SoftEdge','OpenPose'};cbs=[W.Checkbox(description=l,value=l in pre) for l,_ in cn]
urls=W.Textarea(placeholder='Paste SD1.5 checkpoint URLs (.safetensors/.ckpt) one per line',layout=W.Layout(width='100%',height='120px'))
b=W.Button(description='Download Selected',button_style='success',icon='download');out=W.Output()
def click(_):
  out.clear_output()
  with out:
    err=[]
    if vae.value:
      try:download_hf('stabilityai/sd-vae-ft-mse','vae-ft-mse-840000-ema-pruned.safetensors',VAE_DIR)
      except Exception as e:err.append(('VAE',e))
    for cb,(l,r) in zip(cbs,cn):
      if cb.value:
        try:download_hf(r,'diffusion_pytorch_model.safetensors',CN_DIR,f'control_v11p_sd15_{l.lower()}.safetensors')
        except Exception as e:err.append((l,e))
    for u in [u.strip() for u in urls.value.splitlines() if u.strip()]:
      try:n=u.split('/')[-1].split('?')[0]
      except:continue
      if not n.endswith(('.ckpt','.safetensors')):n+='.safetensors'
      try:download_direct(u,CKPT_DIR/n)
      except Exception as e:err.append((u,e))
    print('\nSummary:');[print('✖',n,':',m) for n,m in err] if err else print('✔ Done.')
b.on_click(click);display(Markdown('### Checkpoints & ControlNet')),display(urls,vae,*cbs,b,out)

## Upscalers & Restorers

In [ ]:
from IPython.display import display
def smart(name):
  n=name.lower()
  if 'gfpgan' in n:return GFPGAN_DIR/'GFPGANv1.4.pth'
  if 'codeformer' in n:return CODEFORMER_DIR/'codeformer.pth'
  return ESRGAN_DIR/name
def download_named(url,n):name=n or url.split('/')[-1].split('?')[0];return download_direct(url,smart(name))
ultra=W.Checkbox(description='4x-UltraSharp',value=True);ultra_t=W.Text(placeholder='URL to 4x-UltraSharp.pth',layout=W.Layout(width='100%'))
re=W.Checkbox(description='RealESRGAN_x4plus',value=True);re_t=W.Text(placeholder='URL to RealESRGAN_x4plus.pth',layout=W.Layout(width='100%'))
gfp=W.Checkbox(description='GFPGAN v1.4',value=False);gfp_t=W.Text(placeholder='URL to GFPGANv1.4.pth',layout=W.Layout(width='100%'))
code=W.Checkbox(description='CodeFormer',value=False);code_t=W.Text(placeholder='URL to codeformer.pth',layout=W.Layout(width='100%'))
extra=W.Textarea(placeholder='Extra model URLs (one per line)',layout=W.Layout(width='100%',height='80px'))
b2=W.Button(description='Download Upscalers',button_style='success',icon='download');out2=W.Output()
def go(_):
  out2.clear_output()
  with out2:
    e=[]
    for cb,t,n in [(ultra,ultra_t,'4x-UltraSharp.pth'),(re,re_t,'RealESRGAN_x4plus.pth'),(gfp,gfp_t,'GFPGANv1.4.pth'),(code,code_t,'codeformer.pth')]:
      if cb.value and t.value.strip():
        try:download_named(t.value.strip(),n)
        except Exception as x:e.append((n,x))
    for u in [u.strip() for u in extra.value.splitlines() if u.strip()]:
      try:download_named(u,u.split('/')[-1])
      except Exception as x:e.append((u,x))
    print('Summary:');[print('✖',n,':',x) for n,x in e] if e else print('✔ All done.')
b2.on_click(go);display(ultra,ultra_t,re,re_t,gfp,gfp_t,code,code_t,extra,b2,out2)

## Restart WebUI

In [ ]:
try:
  c='supervisorctl -c /etc/supervisord.conf restart a1111'
  o=subprocess.run(shlex.split(c),capture_output=True,text=True,timeout=20)
  print('✔ Restarted. Refresh WebUI in 10–15 s.' if o.returncode==0 else o.stderr)
except Exception as e:
  print('Manual restart needed:',e)